In [1]:
from jellyfish._jellyfish import damerau_levenshtein_distance
import distance
from sklearn import metrics

In [2]:
from keras.models import load_model
from theano.ifelse import ifelse #added this

Using Theano backend.


In [3]:
import pickle
import pandas as pd
import numpy as np
from utils_test import *

In [4]:
test = pd.read_csv('../input/test.csv')
test = transformDf(test)

In [5]:
with open('../input/parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

In [6]:
with open('../input/preprocessed_data.pkl', "rb") as f:
    X = pickle.load(f)
    y_a = pickle.load(f)
    y_t = pickle.load(f)
    X_test = pickle.load(f)
    y_a_test = pickle.load(f)
    y_t_test = pickle.load(f)

# Utils

In [7]:
model = load_model('output_files/models/model_37-1.50.h5')

In [8]:
test_groupByCase = test.groupby(['CaseID'])

In [9]:
# Check longest case --> suffix: 2 to 7
test_len = findLongestLength(test_groupByCase)
test_len - 5

8

In [10]:
#get features all data
sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4 = getFeature(test_groupByCase)

In [11]:
#get output all data
next_chars, next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4 = getOutput(test_groupByCase)

# All data

## Predict

In [12]:
pred = model.predict(X_test, verbose=0)
pred_a = pred[0]
pred_t = pred[1]

In [13]:
pred[0].shape, pred[1].shape

((4529, 10), (4529, 1))

## Evaluate

### Activity

In [14]:
#for log-loss: get probabilities
gt_a = one_hot_encode(next_chars, targetchartoindice)

In [15]:
metrics.log_loss(gt_a, pred_a)

0.55888155995271493

In [16]:
#for accuracy: get labels
gt_a_label = next_chars
pred_a_label = getLabel(pred_a, targetchartoindice)

In [17]:
metrics.accuracy_score(gt_a_label, pred_a_label)

0.82667255464782508

In [18]:
get_top3_accuracy(pred_a, gt_a_label, targetchartoindice)

0.9916096268491941

In [19]:
# DL distance
#distance.nlevenshtein(gt_a_label, pred_a_label) #0.1731066460587326

In [20]:
#damerau_levenshtein_distance(gt_a_label, pred_a_label) #784

In [21]:
#distance.jaccard(gt_a_label, pred_a_label) #0.4444444444444444

### Time

In [22]:
#get time
gt_t = next_chars_t
pred_t = inverseTime(pred_t, divisor)

In [23]:
metrics.mean_squared_error(gt_t, pred_t)

237771054226.27414

In [24]:
metrics.median_absolute_error(gt_t, pred_t)

19269.94921875

In [25]:
# Convert to day
metrics.mean_absolute_error(gt_t, pred_t)/86400, metrics.median_absolute_error(gt_t, pred_t)/86400

(2.2548319025873695, 0.22303181966145832)

# Suffix

## Predict

In [26]:
suffix = 2 #Change in the interval [2, 7]

In [27]:
sentences_2 = []
sentences_t_2 = []
sentences_t2_2 = []
sentences_t3_2 = []
sentences_t4_2 = []
next_chars_2 = []
next_chars_t_2 = []

for i in range(len(sentences)):
    if len(sentences[i]) == suffix:
        sentences_2.append(sentences[i])
        sentences_t_2.append(sentences_t[i])
        sentences_t2_2.append(sentences_t2[i])
        sentences_t3_2.append(sentences_t3[i])
        sentences_t4_2.append(sentences_t4[i])
        next_chars_2.append(next_chars[i])
        next_chars_t_2.append(next_chars_t[i])

In [28]:
len(sentences_2), len(next_chars_2)

(1268, 1268)

In [29]:
X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)

In [30]:
pred = model.predict(X_test, verbose=0)
pred_a = pred[0]
pred_t = pred[1]

In [31]:
pred[0].shape, pred[1].shape

((1268, 10), (1268, 1))

## Evaluate

### Activity

In [32]:
#for log-loss: get probabilities
gt_a = one_hot_encode(next_chars_2, targetchartoindice)

In [33]:
metrics.log_loss(gt_a, pred_a)

0.666694794149852

In [34]:
#for accuracy: get label
gt_a_label = next_chars_2
pred_a_label = getLabel(pred_a, targetchartoindice)

In [35]:
metrics.accuracy_score(gt_a_label, pred_a_label)

0.77839116719242907

In [36]:
get_top3_accuracy(pred_a, gt_a_label, targetchartoindice)

0.9873817034700315

In [37]:
# DL distance
distance.nlevenshtein(gt_a_label, pred_a_label)

0.22082018927444794

In [38]:
damerau_levenshtein_distance(gt_a_label, pred_a_label)

280

In [39]:
distance.jaccard(gt_a_label, pred_a_label)

0.5

### Time

In [40]:
#get time
gt_t = next_chars_t_2
pred_t = inverseTime(pred_t, divisor)

In [41]:
metrics.mean_squared_error(gt_t, pred_t)

346547304890.93604

In [42]:
metrics.mean_absolute_error(gt_t, pred_t), metrics.median_absolute_error(gt_t, pred_t)

(278004.10952437337, 32249.7021484375)

In [43]:
# Convert to day
metrics.mean_absolute_error(gt_t, pred_t)/86400, metrics.median_absolute_error(gt_t, pred_t)/86400

(3.2176401565320991, 0.37326044153284144)